In [1]:
import pandas as pd
import sqlite3

In [2]:
class WeatherDatabase:
    url: str

    def __init__(self, url: str, *args):
        self.url = url
        con = sqlite3.connect(url)
        cur = con.cursor()
        for table in args:
            con.execute(
                f"""
                CREATE DATABASE IF NOT EXISTS Expences
                """)
        cur.close()
        con.commit()
        con.close()

    def call_db(self, query, *args):
        con = sqlite3.connect(self.url)
        cur = con.cursor()
        res = cur.execute(query, args)
        data = res.fetchall()
        cur.close()
        con.commit()
        con.close()
        return data

In [48]:
db = WeatherDatabase('./WeatherGBG.db')

In [66]:
query = """SELECT *, ROUND(AVG(Lufttemperatur), 2) AS Avg_temp
            FROM Temp_Save_raw
            GROUP BY Datum"""


In [67]:
conn = sqlite3.connect('./WeatherGBG.db')

In [68]:
df_avg_temp_daily = pd.read_sql_query(query, conn)

In [69]:
df_avg_temp_daily = df_avg_temp_daily.drop('id', axis=1)

In [70]:
df_avg_temp_daily = df_avg_temp_daily.drop(['Lufttemperatur', 'Tid_UTC'], axis=1)

In [71]:
df_avg_temp_daily

,Datum,Kvalitet,Avg_temp
0,1944-01-01,G,3.20
1,1944-01-02,G,3.60
2,1944-01-03,G,2.80
3,1944-01-04,G,-4.47
4,1944-01-05,G,-1.40
...,...,...,...
22980,2006-12-01,G,8.03
22981,2006-12-02,G,8.81
22982,2006-12-03,G,8.11
22983,2006-12-04,G,8.57


In [72]:
# Compare to previous dataframe
pd.read_pickle('../Dataframes/df_daily_temp.pkl')

,Datum,Kvalitet,Snittemperatur,Year
0,1944-01-01,G,3.20,1944-01-01
1,1944-01-02,G,3.60,1944-01-01
2,1944-01-03,G,2.80,1944-01-01
3,1944-01-04,G,-4.47,1944-01-01
4,1944-01-05,G,-1.40,1944-01-01
...,...,...,...,...
29109,2023-10-12,G,12.08,2023-01-01
29110,2023-10-13,G,11.89,2023-01-01
29111,2023-10-14,G,11.54,2023-01-01
29112,2023-10-15,G,10.09,2023-01-01


In [105]:
month_year_query = """SELECT *,
                        ROUND(AVG(Lufttemperatur), 2) AS Avg_temp,
                        SUBSTR(Datum, 1, 4) AS Year,
                        SUBSTR(Datum, 6, 2) AS Month
            FROM Temp_Save_raw
            GROUP BY Datum"""


In [106]:
df = pd.read_sql_query(month_year_query, conn)
df.drop('id', axis=1)

,Datum,Tid_UTC,Lufttemperatur,Kvalitet,Avg_temp,Year,Month
0,1944-01-01,07:00:00,1.6,G,3.20,1944,01
1,1944-01-02,07:00:00,0.6,G,3.60,1944,01
2,1944-01-03,07:00:00,3.8,G,2.80,1944,01
3,1944-01-04,07:00:00,-3.4,G,-4.47,1944,01
4,1944-01-05,07:00:00,-6.8,G,-1.40,1944,01
...,...,...,...,...,...,...,...
22980,2006-12-01,00:00:00,8.3,G,8.03,2006,12
22981,2006-12-02,00:00:00,9.2,G,8.81,2006,12
22982,2006-12-03,00:00:00,8.5,G,8.11,2006,12
22983,2006-12-04,00:00:00,9.3,G,8.57,2006,12


In [125]:
last_date = df[-1:]['Datum'].values[0]

In [126]:
last_date

'2006-12-05'

In [130]:
month_year_query_gbg = """SELECT *,
                        ROUND(AVG(Lufttemperatur), 2) AS Avg_temp,
                        SUBSTR(Datum, 1, 4) AS Year,
                        SUBSTR(Datum, 6, 2) AS Month
            FROM Temp_GBG_raw
            WHERE Datum > 2006-12-05
            GROUP BY Datum"""

In [131]:
df = pd.read_sql_query(month_year_query_gbg, conn)
df.drop('id', axis=1)

,Datum,Tid_UTC,Lufttemperatur,Kvalitet,Avg_temp,Year,Month
0,1961-01-01,06:00:00,0.8,G,1.07,1961,01
1,1961-01-02,06:00:00,1.8,G,2.27,1961,01
2,1961-01-03,06:00:00,2.6,G,2.27,1961,01
3,1961-01-04,06:00:00,2.8,G,2.53,1961,01
4,1961-01-05,06:00:00,1.8,G,2.13,1961,01
...,...,...,...,...,...,...,...
19909,2023-06-27,00:00:00,16.3,G,18.18,2023,06
19910,2023-06-28,00:00:00,16.2,G,19.73,2023,06
19911,2023-06-29,00:00:00,16.8,G,19.23,2023,06
19912,2023-06-30,00:00:00,19.2,G,17.40,2023,06


In order sort by dates, the dates must be in date-format
To do this, create a new column that has the dates in a date-format, see below

In [132]:
add_colum = """ALTER TABLE Temp_GBG_raw
                ADD COLUMN Datum_date DATE"""

In [133]:
db.call_db(add_colum)

[]

In [134]:
add_dates_to_column = """UPDATE Temp_GBG_raw
                            SET Datum_date = DATE(Datum)"""

In [135]:
db.call_db(add_dates_to_column)

[]

In [136]:
add_colum = """ALTER TABLE Temp_Save_raw
                ADD COLUMN Datum_date DATE"""
db.call_db(add_colum)
add_dates_to_column = """UPDATE Temp_Save_raw
                            SET Datum_date = DATE(Datum)"""
db.call_db(add_dates_to_column)

[]

In [139]:
# Precipitation for GBG

add_colum_gbg = """ALTER TABLE precipitation_data_GBG_raw
                ADD COLUMN Datum_date DATE"""
db.call_db(add_colum_gbg)
add_dates_to_column_gbg = """UPDATE precipitation_data_GBG_raw
                            SET Datum_date = DATE(Representativt_dygn)"""
db.call_db(add_dates_to_column_gbg)

[]

In [140]:
# Precipitation for Säve

add_colum_save = """ALTER TABLE precipitation_data_save_raw
                ADD COLUMN Datum_date DATE"""
db.call_db(add_colum_save)
add_dates_to_column_save = """UPDATE precipitation_data_save_raw
                            SET Datum_date = DATE(Representativt_dygn)"""
db.call_db(add_dates_to_column_save)

[]

Add date column for precipitation

In [73]:
# for ind, val in df.iterrows():
#     print(ind)
#     print(val)
#     break